# 내용 총정리?

`-` 베이지안 일반

* 베이즈 정리

$$P(Z~|~X) = \frac{P(X~|~Z)P(Z)}{P(X)}$$

> 관측된 $X$를 이용하여 확률적으로 $Z$를 알아내는 것.
>
> 처음부터 Scalar/Vector 값을 목표로 하는 게 아니라, 확률 자체를 알아내고자 하는 것 $\to$ 불확실성에 대한 이야기를 기본적으로 할 수 있다는 특징
>
> Likelihood $\times$ Prior $\propto$ Posterior. Prior가 적지않은 영향을 미침
>
> Evidence $p(X)$의 값이 크기 위해서는 분자의 확률값이 커야 함. Likelihood와 Prior에 의존하는 값으로 나타남 $\to$ 모형과 Prior가 데이터에 적합한지를 확인하는 척도. ELBO를 유도해서 여러가지 알고리즘을 개발할 수 있었음.

* 복잡한 모델에서 탄탄한 수학적 배경을 기반으로 불확실성을 설명 가능
* 베이지안 개념 자체가 좋긴 하지만, 자원의 문제가 있기 때문에 VAE를 사용해서 근사할 필요가 있었음.

`-` VAE

* $Z~|~X$로 확률을 모델링하여 생성까지... 인코더를 만드는 게 쉽지 않음

$$\mathbb E_{q_{\phi}} [\log p_{\theta}(x^{(i)}~|~z)] - \text{KL}(q~||~p)$$

* 재OO화? Reparametrization Trick
* $\log p_{\theta}$는 Loss Function이라고 보면 되는데, KL divergence는 손으로 구할 수 있는 수식이며 PDF가 필요함. 샘플링된 값이 아니라 분포 자체가 필요
* Denoising은 잘하지만, Fine Image가 들어왔을 때 노이즈를 넣음
* l2 loss같이 mean을 따라가도록 만드는 손실을 거치기 때문에, 모형의 구조 상 노이즈가 무조건 존재한다고 생각하고 오차를 제거하고자 이미지를 뭉갬. 블러 현상은 피할 수 없음

`-` MC Dropout

* 뉴런 연결을 일부 해제하는 특수한 형태의 앙상블...
* 독립적이지 않은 모델들로 이뤄진 앙상블
* 그냥 Dropout과 달리, 추론 단계에서 훈련 상태를 유지시키고 MC sampling으로 불확실성 계산
> VAE의 정확한 솔루션이다. VAE의 발전된 버전. 수학적 배경 견고함.

* Variational Bayes는 독립인 정규분포로의 근사가 기본이 됨
* MC Dropout은 정규분포를 쓰는 게 아니라, 베르누이 분포에 상수를 곱한 것을 Prior로 사용 $\to$ 최종 $\hat M$은 dropout으로 얻을 수 있음
* 성능이 생각보다 잘 안나오고, 데이터에 따라 결과가 많이 달라짐

`-` Gaussian Process

* Random Function. 커널과 평균 벡터로 결정
* 다변량 정규분포의 성질에 기반. 조건부 분포도 Normal이라는 것에 의거
* 공분산은 최소 PSD. $\to$ RBF, Matern, non-stationary...
* 추론 시 커널의 diagonal term에 분산을 더해줘야 함 (epsilon term)
* 관측된 값은 band의 너비가 좁으나, 관측되지 않은 값은 band의 너비가 넓음. 하지만 GT로 완벽하게 수렴하진 않음 ($f$ 추정의 문제가 아니라 $y$ 추정의 문제임. 모델에 오차가 있기 때문)
> Prediction Interval: $y$가 해당 구간에 있을 확률을 규정

* Confidence Interval: Randomness가 들어가지 않은 모수에 대한 구간 ($\theta, f$)
* Predictive Interval: 실제로 관측이 되는 변량들에 대한 구간 ($y$)
* Credible Interval: 확률변수들의 구간 ($\theta, f, y$. 위 두가지를 모두 포함. 모두를 확률변수라고 보기 때문.)

`-` Laplace Approximation

* Classification에서 GP 사용 가능하도록 근사
* VB는 근사 시 모든 객체들이 독립이라고 가정하지만, LA는 Covariance matrix를 잡음 $\to$ 독립인 Covariance matrix가 나올 수 없음
* VB는 불확실성 측정에 문제가 있음(과소추정). 평균은 문제가 없음. Correlation이 강할 때에는 문제가 심해짐.

`-` Uncertainty

**Regression**

$$y = f + \epsilon$$

* $\epsilon$ 때문에 발생하는 Uncertainty
> 줄일 수 없음
* $\hat f$ 추정에서 발생하는 Uncertainty: $\text{Var}(\hat f) > 0$
> 모델을 잘 설정하여 줄일 수 있음

**Classification**

$$\log \frac{p}{1-p} = f(x)$$

* $\hat f$ 추정의 Uncertainty는 줄일 수 있음
* 확률값 자체로 인한 Uncertainty. Confidence가 없음에 따르는 불확실성. 불가피함. $p$를 정확히 알아낸다고 해도 확률 구조 자체에 대한 불확실성이 필연적

* $\epsilon$ 때문에 발생하는 분산 자체를 컨트롤할 수는 없지만, 모델을 잘 설정함으로써 GT에 가깝게 줄일 수 있음. 이걸 너무 줄이게 되면 Overfitting